# **Applicatons of RNNs - Natural Language Prediction**
 * # We now build a  Recurrent Neural Network in Keras  to predict next character in the the Arthur Conan Doyle's book "The Adventures of Sherlock Holmes"

In [1]:
import numpy as np
import keras
import os
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, Dense, Input, Bidirectional
#from keras.layers.core import  Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [2]:
!curl -O https://www.gutenberg.org/files/1661/1661-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  593k  100  593k    0     0   278k      0  0:00:02  0:00:02 --:--:--  278k


In [3]:
path = '1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print(f"text[:100]:{text[:100]}")
chars = sorted(list(set(text)))
print("Total distinct chars:", len(chars))
# char dictionaries
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 581425
text[:100]:﻿the project gutenberg ebook of the adventures of sherlock holmes, by arthur conan doyle  this ebook
Total distinct chars: 71


In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [7]:
%cd gdrive/MyDrive/lecture-13b

/content/gdrive/MyDrive/lecture-13b


In [8]:
%ls

NCP5_model.weights.h5


# **Create sequences/sentencies of 40 characters and predict the next one**

In [9]:
# cut the text in overlapping sequences of window_size characters
# each time we slide of one character
window_size = 40

sentences = []
next_chars = []
for i in range(len(text) - window_size):
    sentences.append(text[i : i + window_size])
    next_chars.append(text[i + window_size])

print("Number of sequences:", len(sentences))

Number of sequences: 581385


# **Character encoding: one-hot**
 * # Again, better and more efficient encoding can be defined
 * # Here we do not focus on performance

In [10]:
# Creating one hot encoding
x = np.zeros((len(sentences), window_size, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [11]:
print(x.shape, y.shape)

(581385, 40, 71) (581385, 71)


# **Build the model: a single LSTM layer**

In [ ]:
model_file1 = 'NCP1_model.weights.h5'
import os
if os.path.isfile(model_file1):
  print("file exists, loading")
  model1 = load_model(model_file1)
else:
  model1 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(128),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model1.summary())

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_27 (LSTM)                  │ (None, 128)            │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 71)             │         9,159 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,559 (435.78 KB)

 Trainable params: 111,559 (435.78 KB)

 Non-trainable params: 0 (0.00 B)

None


# **Going deeper: multiple LSTM hidden layers**
 * # Need to return output sequences from previous layers
* # can be done by setting argument ```return_sequences=True```  

In [ ]:
import os
model_file2 = 'NCP2_model.weights.h5'

if os.path.isfile(model_file2):
  print("file exists, loading")
  model2 = load_model(model_file2)
else:
  model2 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(82, return_sequences=True),# deep RNN
        LSTM(82),
        Dense(len(chars), activation="softmax")
    ]
  )

print(model2.summary())

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_28 (LSTM)                  │ (None, 40, 82)         │        50,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_29 (LSTM)                  │ (None, 82)             │        54,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 71)             │         5,893 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,525 (431.74 KB)

 Trainable params: 110,525 (431.74 KB)

 Non-trainable params: 0 (0.00 B)

None


# **Constructing bidirectional RNN**
 * # In keras can be done by using the ```Bidirectional``` layer

In [ ]:
model_file3 = 'NCP3_model.weights.h5'
if os.path.isfile(model_file3):
  print("file exists, loading")
  model3 = load_model(model_file3)
else:
  model3 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        Bidirectional(LSTM(82)),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model3.summary())

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional) │ (None, 164)            │       101,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 71)             │        11,715 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,739 (440.39 KB)

 Trainable params: 112,739 (440.39 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model_file4 = 'NCP4_model.weights.h5'

if os.path.isfile(model_file4):
  print("file exists, loading")
  model4 = load_model(model_file4)
else:
  model4 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        GRU(150),
        Dense(len(chars), activation="softmax"),
    ]
  )

print(model4.summary())

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 150)            │       100,350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 71)             │        10,721 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,071 (433.87 KB)

 Trainable params: 111,071 (433.87 KB)

 Non-trainable params: 0 (0.00 B)

None



# **Training the models**

In [ ]:
if not os.path.isfile(model_file1):
  checkpoint = ModelCheckpoint(model_file1, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 512
  model1.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            callbacks=callbacks_list,
            validation_split=0.1)
  model1.save_weights(model_file1)

Epoch 1/10
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2358 - loss: 2.7975

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.2359 - loss: 2.7969 - val_accuracy: 0.3412 - val_loss: 2.3074
Epoch 2/10
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3678 - loss: 2.1559

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.3678 - loss: 2.1559 - val_accuracy: 0.3726 - val_loss: 2.1852
Epoch 3/10
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4005 - loss: 2.0234

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.4006 - loss: 2.0234 - val_accuracy: 0.3954 - val_loss: 2.1049
Epoch 4/10
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4245 - loss: 1.9424

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.4245 - loss: 1.9424 - val_accuracy: 0.4134 - val_loss: 2.0452
Epoch 5/10
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4404 - loss: 1.8801

...

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4506 - loss: 1.8398 - val_accuracy: 0.4360 - val_loss: 1.9614
Epoch 7/10
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4635 - loss: 1.7919

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4635 - loss: 1.7919 - val_accuracy: 0.4467 - val_loss: 1.9292
Epoch 8/10
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4722 - loss: 1.7560

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.4722 - loss: 1.7560 - val_accuracy: 0.4553 - val_loss: 1.9043
Epoch 9/10
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4821 - loss: 1.7230

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.4821 - loss: 1.7230 - val_accuracy: 0.4616 - val_loss: 1.8849
Epoch 10/10
1019/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4909 - loss: 1.6942

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.4909 - loss: 1.6941 - val_accuracy: 0.4665 - val_loss: 1.8619


In [ ]:
if not os.path.isfile(model_file2):
  checkpoint = ModelCheckpoint(model_file2, monitor='val_loss', verbose=0,
                             save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 512
  model2.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)

  model2.save_weights(model_file2)

Epoch 1/20
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2154 - loss: 2.9220

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - accuracy: 0.2155 - loss: 2.9215 - val_accuracy: 0.3094 - val_loss: 2.4194
Epoch 2/20
1019/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3310 - loss: 2.2701

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.3310 - loss: 2.2699 - val_accuracy: 0.3514 - val_loss: 2.2750
Epoch 3/20
1019/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3735 - loss: 2.1285

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.3735 - loss: 2.1284 - val_accuracy: 0.3731 - val_loss: 2.1876
Epoch 4/20
1019/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3957 - loss: 2.0367

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.3957 - loss: 2.0366 - val_accuracy: 0.3868 - val_loss: 2.1214
Epoch 5/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4123 - loss: 1.9689

...

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.4996 - loss: 1.6645 - val_accuracy: 0.4721 - val_loss: 1.8491
Epoch 17/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5029 - loss: 1.6511

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.5029 - loss: 1.6511 - val_accuracy: 0.4748 - val_loss: 1.8367
Epoch 18/20
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5067 - loss: 1.6396

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.5067 - loss: 1.6396 - val_accuracy: 0.4784 - val_loss: 1.8306
Epoch 19/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5094 - loss: 1.6280

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.5094 - loss: 1.6280 - val_accuracy: 0.4781 - val_loss: 1.8263
Epoch 20/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5141 - loss: 1.6134

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.5141 - loss: 1.6134 - val_accuracy: 0.4824 - val_loss: 1.8171


In [ ]:
if not os.path.isfile(model_file3):
  checkpoint = ModelCheckpoint(model_file3, monitor='val_loss', verbose=0,
                              save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  checkpoint = ModelCheckpoint(model_file3, monitor='val_loss', verbose=0,
                              save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 512
  model3.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)

  model3.save_weights(model_file3)

Epoch 1/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2391 - loss: 2.7940

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - accuracy: 0.2393 - loss: 2.7931 - val_accuracy: 0.3488 - val_loss: 2.2915
Epoch 2/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3789 - loss: 2.1205

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.3789 - loss: 2.1204 - val_accuracy: 0.3871 - val_loss: 2.1498
Epoch 3/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4136 - loss: 1.9836

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.4136 - loss: 1.9836 - val_accuracy: 0.4140 - val_loss: 2.0605
Epoch 4/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4377 - loss: 1.8984

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.4377 - loss: 1.8983 - val_accuracy: 0.4265 - val_loss: 1.9988
Epoch 5/20
1019/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4566 - loss: 1.8282

...

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.5345 - loss: 1.5400 - val_accuracy: 0.4950 - val_loss: 1.7588
Epoch 17/20
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5382 - loss: 1.5251

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.5382 - loss: 1.5251 - val_accuracy: 0.4986 - val_loss: 1.7495
Epoch 18/20
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5427 - loss: 1.5154

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5427 - loss: 1.5154 - val_accuracy: 0.4995 - val_loss: 1.7424
Epoch 19/20
1020/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5457 - loss: 1.5048

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5457 - loss: 1.5048 - val_accuracy: 0.5014 - val_loss: 1.7371
Epoch 20/20
1021/1022 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5487 - loss: 1.4911

1022/1022 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5487 - loss: 1.4911 - val_accuracy: 0.5030 - val_loss: 1.7325


In [ ]:
if not os.path.isfile(model_file4):
  checkpoint = ModelCheckpoint(model_file4, monitor='val_loss', verbose=0,
                              save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 256
  model4.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)

  model4.save_weights(model_file4)

Epoch 1/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3109 - loss: 2.4561

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - accuracy: 0.3109 - loss: 2.4559 - val_accuracy: 0.4278 - val_loss: 1.9761
Epoch 2/20
2042/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4738 - loss: 1.7632

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.4738 - loss: 1.7631 - val_accuracy: 0.4795 - val_loss: 1.7978
Epoch 3/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5216 - loss: 1.5919

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.5216 - loss: 1.5919 - val_accuracy: 0.5020 - val_loss: 1.7160
Epoch 4/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5497 - loss: 1.4932

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.5497 - loss: 1.4931 - val_accuracy: 0.5143 - val_loss: 1.6710
Epoch 5/20
2037/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5674 - loss: 1.4283

...

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.5997 - loss: 1.3091 - val_accuracy: 0.5452 - val_loss: 1.6012
Epoch 10/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6060 - loss: 1.2816

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.6060 - loss: 1.2816 - val_accuracy: 0.5474 - val_loss: 1.5968
Epoch 11/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6091 - loss: 1.2733

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6091 - loss: 1.2733 - val_accuracy: 0.5456 - val_loss: 1.5967
Epoch 12/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6127 - loss: 1.2567 - val_accuracy: 0.5480 - val_loss: 1.6011
Epoch 13/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.6159 - loss: 1.2444 - val_accuracy: 0.5440 - val_loss: 1.6009
Epoch 14/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6190 - loss: 1.2337

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.6190 - loss: 1.2337 - val_accuracy: 0.5478 - val_loss: 1.5948
Epoch 15/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6217 - loss: 1.2236

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.6217 - loss: 1.2236 - val_accuracy: 0.5435 - val_loss: 1.5936
Epoch 16/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.6228 - loss: 1.2184 - val_accuracy: 0.5478 - val_loss: 1.5944
Epoch 17/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.6265 - loss: 1.2062 - val_accuracy: 0.5505 - val_loss: 1.6039
Epoch 18/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.6276 - loss: 1.1990 - val_accuracy: 0.5501 - val_loss: 1.6056
Epoch 19/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.6293 - loss: 1.1956 - val_accuracy: 0.5482 - val_loss: 1.6050
Epoch 20/20
2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.6303 - loss: 1.1892 - val_accuracy: 0.5478 - val_loss: 1.6097


# Function to sample next character

In [12]:
# given the predictite probability distribution pred, we need to sample the
#   the next character
#  We use a scheme used in thermodynamics named TEMPERATURE SAMPLING
# where high temperature means low energy states are more likely encountered.
# In probability models, logits play the role of energy and we can implement
# temperature sampling by dividing logits by the temperature before feeding
#    them into softmax and obtaining our sampling probabilities

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # The multinomial distribution is a multivariate generalization of
    #   the binomial distribution. Take an experiment with one of p possible
    #    outcomes
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Regenerating the text

In [ ]:
# number of characters to generate
to_be_gen = 50
temperature = 1
generated = ''
print(f"text:{text[0:to_be_gen+window_size]}")
for i in range(to_be_gen):
    start_index = i
    sentence = text[start_index : start_index + window_size]

    x_pred = np.zeros((1, window_size, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model1.predict(x_pred, verbose=0)[0]
#    next_index = sample(preds, temperature)
    next_index = np.argmax(preds)
#    next_word = list(word_dict.keys())[list(word_dict.values()).index(next_index)]
    next_char = indices_char[next_index]
    print(f"input: {sentence}, \t pred:{next_char}")
    sentence = sentence[1:] + next_char
    generated += next_char

print("...Generated: ", generated)


# **Exercise**
 * # Choose a model among those above and evaluate what is better between:
* # having smaller states but more hidden layers
* # having larger states but less hidden layers
* # almost equal the number of parameters
* # **Increase the number of epochs at least to 20/30**  

In [26]:
# Smaller states but more hidden layers:
model5 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(50, return_sequences=True), #deep RNN
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(len(chars), activation="softmax")
    ]
)

print(model5.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_11 (LSTM)                  │ (None, 40, 50)         │        24,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 40, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 71)             │         3,621 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,421 (267.27 KB)

 Trainable params: 68,421 (267.27 KB)

 Non-trainable params: 0 (0.00 B)

None


In [27]:
# Larger states but less hidden layers
model6 = keras.Sequential(
    [
        Input(shape=(window_size, len(chars))),
        LSTM(93),  # larger state, only one layer
        Dense(len(chars), activation="softmax")
    ]
)

print(model6.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_14 (LSTM)                  │ (None, 93)             │        61,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 71)             │         6,674 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,054 (265.84 KB)

 Trainable params: 68,054 (265.84 KB)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
# Train model5 ( - states + hidden layers)
model_file5 = 'NCP5_model.weights.h5'
if not os.path.isfile(model_file5):
  checkpoint = ModelCheckpoint(model_file5, monitor='val_loss', verbose=0,
                              save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 256
  model5.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)

  model5.save_weights(model_file5)

Epoch 1/20
2039/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2122 - loss: 2.9082

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step - accuracy: 0.2123 - loss: 2.9074 - val_accuracy: 0.2983 - val_loss: 2.4225
Epoch 2/20
2039/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3271 - loss: 2.2849

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - accuracy: 0.3272 - loss: 2.2848 - val_accuracy: 0.3576 - val_loss: 2.2753
Epoch 3/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3792 - loss: 2.1195

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - accuracy: 0.3793 - loss: 2.1194 - val_accuracy: 0.3862 - val_loss: 2.1882
Epoch 4/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4065 - loss: 2.0245

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - accuracy: 0.4065 - loss: 2.0244 - val_accuracy: 0.4008 - val_loss: 2.1271
Epoch 5/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4186 - loss: 1.9674

...

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 42s 12ms/step - accuracy: 0.4959 - loss: 1.6900 - val_accuracy: 0.4693 - val_loss: 1.8810
Epoch 17/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4991 - loss: 1.6792

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.4991 - loss: 1.6792 - val_accuracy: 0.4696 - val_loss: 1.8701
Epoch 18/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5026 - loss: 1.6679

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.5026 - loss: 1.6679 - val_accuracy: 0.4695 - val_loss: 1.8661
Epoch 19/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5058 - loss: 1.6541

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.5058 - loss: 1.6541 - val_accuracy: 0.4730 - val_loss: 1.8579
Epoch 20/20
2042/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5082 - loss: 1.6468

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - accuracy: 0.5082 - loss: 1.6468 - val_accuracy: 0.4720 - val_loss: 1.8549


In [29]:
# Train model6 ( + states - hidden layers)
model_file6 = 'NCP6_model.weights.h5'

if not os.path.isfile(model_file6):
  checkpoint = ModelCheckpoint(model_file6, monitor='val_loss', verbose=0,
                              save_best_only=True, mode='min')

  callbacks_list = [checkpoint]
  model6.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-3),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  epochs = 20
  batch_size = 256
  model6.fit(x, y,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,callbacks=callbacks_list,
            validation_split=0.1)

  model6.save_weights(model_file6)

Epoch 1/20
2038/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2639 - loss: 2.6702

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.2641 - loss: 2.6692 - val_accuracy: 0.3673 - val_loss: 2.2296
Epoch 2/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3931 - loss: 2.0637

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.3931 - loss: 2.0636 - val_accuracy: 0.3986 - val_loss: 2.0946
Epoch 3/20
2038/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4274 - loss: 1.9314

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.4274 - loss: 1.9313 - val_accuracy: 0.4208 - val_loss: 2.0096
Epoch 4/20
2043/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4505 - loss: 1.8470

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.4505 - loss: 1.8470 - val_accuracy: 0.4413 - val_loss: 1.9546
Epoch 5/20
2037/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4672 - loss: 1.7847

...

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.5425 - loss: 1.5147 - val_accuracy: 0.5033 - val_loss: 1.7352
Epoch 17/20
2042/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5463 - loss: 1.4998

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.5463 - loss: 1.4998 - val_accuracy: 0.5044 - val_loss: 1.7254
Epoch 18/20
2042/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5492 - loss: 1.4905

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5492 - loss: 1.4904 - val_accuracy: 0.5063 - val_loss: 1.7173
Epoch 19/20
2040/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5516 - loss: 1.4814

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.5516 - loss: 1.4814 - val_accuracy: 0.5096 - val_loss: 1.7154
Epoch 20/20
2036/2044 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5543 - loss: 1.4726

2044/2044 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.5543 - loss: 1.4726 - val_accuracy: 0.5104 - val_loss: 1.7070


In [30]:
# for model5
to_be_gen = 50
temperature = 1
generated = ''
print(f"text:{text[0:to_be_gen+window_size]}")
for i in range(to_be_gen):
    start_index = i
    sentence = text[start_index : start_index + window_size]

    x_pred = np.zeros((1, window_size, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model5.predict(x_pred, verbose=0)[0]
#    next_index = sample(preds, temperature)
    next_index = np.argmax(preds)
#    next_word = list(word_dict.keys())[list(word_dict.values()).index(next_index)]
    next_char = indices_char[next_index]
    print(f"input: {sentence}, \t pred:{next_char}")
    sentence = sentence[1:] + next_char
    generated += next_char

print("...Generated: ", generated)

text:﻿the project gutenberg ebook of the adventures of sherlock holmes, by arthur conan doyle  
input: ﻿the project gutenberg ebook of the adve, 	 pred:r
input: the project gutenberg ebook of the adven, 	 pred:t
input: he project gutenberg ebook of the advent, 	 pred:i
input: e project gutenberg ebook of the adventu, 	 pred:r
input:  project gutenberg ebook of the adventur, 	 pred:e
input: project gutenberg ebook of the adventure, 	 pred: 
input: roject gutenberg ebook of the adventures, 	 pred: 
input: oject gutenberg ebook of the adventures , 	 pred:o
input: ject gutenberg ebook of the adventures o, 	 pred:f
input: ect gutenberg ebook of the adventures of, 	 pred: 
input: ct gutenberg ebook of the adventures of , 	 pred:t
input: t gutenberg ebook of the adventures of s, 	 pred:h
input:  gutenberg ebook of the adventures of sh, 	 pred:e
input: gutenberg ebook of the adventures of she, 	 pred: 
input: utenberg ebook of the adventures of sher, 	 pred:l
input: tenberg ebook of the advent

In [31]:
# for model6
to_be_gen = 50
temperature = 1
generated = ''
print(f"text:{text[0:to_be_gen+window_size]}")
for i in range(to_be_gen):
    start_index = i
    sentence = text[start_index : start_index + window_size]

    x_pred = np.zeros((1, window_size, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model6.predict(x_pred, verbose=0)[0]
#    next_index = sample(preds, temperature)
    next_index = np.argmax(preds)
#    next_word = list(word_dict.keys())[list(word_dict.values()).index(next_index)]
    next_char = indices_char[next_index]
    print(f"input: {sentence}, \t pred:{next_char}")
    sentence = sentence[1:] + next_char
    generated += next_char

print("...Generated: ", generated)

text:﻿the project gutenberg ebook of the adventures of sherlock holmes, by arthur conan doyle  
input: ﻿the project gutenberg ebook of the adve, 	 pred:r
input: the project gutenberg ebook of the adven, 	 pred:t
input: he project gutenberg ebook of the advent, 	 pred:i
input: e project gutenberg ebook of the adventu, 	 pred:r
input:  project gutenberg ebook of the adventur, 	 pred:e
input: project gutenberg ebook of the adventure, 	 pred: 
input: roject gutenberg ebook of the adventures, 	 pred: 
input: oject gutenberg ebook of the adventures , 	 pred:w
input: ject gutenberg ebook of the adventures o, 	 pred:f
input: ect gutenberg ebook of the adventures of, 	 pred: 
input: ct gutenberg ebook of the adventures of , 	 pred:t
input: t gutenberg ebook of the adventures of s, 	 pred:o
input:  gutenberg ebook of the adventures of sh, 	 pred:e
input: gutenberg ebook of the adventures of she, 	 pred: 
input: utenberg ebook of the adventures of sher, 	 pred:l
input: tenberg ebook of the advent